In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc, FREQS
import plot as plot
import slices as slices
from ionfrac import VMIN, VMAX
import projs as projs
# import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
Snap054_n = snapshot(snap=54, name = 'Snap054_n', read_zeusmp=False)
Snap054_n.load_shock_variables()

nums = np.array([59, 60, 61, 62, 63])
years = np.array(['54,050 yr', '54,060 yr', '54,070 yr', '54,080 yr', '54,090 yr'])
zoomslicescales = np.array([500,4000])

snaps = np.empty_like(nums, dtype=basic_snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = basic_snapshot(snap=num, name = ('Snap%03d_n54' % num))
    snaps[ii].x1 = Snap054_n.x1
    snaps[ii].x2 = Snap054_n.x2
    snaps[ii].x3 = Snap054_n.x3
    # snaps[ii].load_shock_variables()
    # snaps[ii].load_fluxes()
    # snaps[ii].load_intensity_variables()

In [ ]:
freqs = FREQS

# r_kpc = 1
# heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
# scales = heights_and_scales['scales'] # AU
# heights = heights_and_scales['heights'] # AU  

# Intensity Map Methods

In [ ]:
freq_strings=np.array(['.01', '.05', '0.1', '0.5', '1.0', 
                       '5.3', '23', '43', '100', '230']) 

def zoom_intensity_pcolormesh(Snap, year, scale, f, vmin, vmax, cmap='CMRmap',
                        saveloc=False, show_cbar= False, show_xlabels=True, show_ylabels=True, cbar_pad=.2,
                             vertical_cbar=False):
    nu=freqs[f]
    Snap.load_intensity_variables(nu)
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')

    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.InuA[:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    ax.vlines(x=(-250,250), ymin=0, ymax=500, colors='white')
    ax.hlines(y=(0,500), xmin=-250, xmax=250, colors='white')
    
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.tick_params(axis='x', labelsize=20)
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 14)
            cbar.ax.tick_params(rotation=45, labelsize = 13)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize=14)
            cbar.ax.tick_params(rotation=45, labelsize=13)
    ax.text(.02, 1, (freq_strings[f]+'GHz'), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 25, color='white', weight='bold')
    ax.text(.02, 1, ('\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, color='white', weight='bold')
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_'+str(scale)+'AU.png' 
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return(filename)

In [ ]:
# ZoomSnapshots = np.array([Snap059_hf, Snap060_hf, Snap061_hf, Snap062_hf, Snap063_hf])
# ZoomYears = np.array(['54,050 yrs', '54,060 yrs', '54,070 yrs', '54,080 yrs', '54,090 yrs'])
ZoomSnapshots =  snaps
ZoomYears = years
zoomslicescales = np.array([500,4000])


nrows = 2
ncols = 5
scale = zoomslicescales[0]
zoomfiles = np.empty((nrows, ncols), dtype=object)
saveloc = VICO_loc+'/pillowfiles/zoom_12msun_fig'
vmin_zoom, vmax_zoom = -2,5
f=9

r=0
for scale in zoomslicescales:
    c=0
    for s in range(len(ZoomSnapshots)):
        # print(ZoomYears[s])
        if(r==0): xlabels = False
        else: xlabels = True
        if(c==0): ylabels = True
        else: ylabels = False
        filename = zoom_intensity_pcolormesh(ZoomSnapshots[s],
                            ZoomYears[s], scale, f=9,
                            vmin = vmin_zoom, vmax = vmax_zoom,
                            show_xlabels=xlabels, show_ylabels=ylabels,
                                        saveloc=saveloc)
        # print(filename)
        zoomfiles[r,c] = filename
        img = Image.open(filename)
        # print(img.size)
        c+=1
    r+=1


In [ ]:
allimage, details = plot.pil_image(zoomfiles)
display(allimage)

In [ ]:
# for PIL    
def make_zoom_allimage(imgfiles_arr):
    nrows = len(imgfiles_arr)
    ncols = len(imgfiles_arr[0]) # should be 5 for 
    side=300
    header=0
    leftax = 135
    bottomax = int((26+58)*side/231)
    allimage=Image.new("RGBA", (ncols*side + leftax, 
                                nrows*side + bottomax + header))
    cropped_names = np.empty((nrows, ncols), dtype=object)

    for r in range(nrows):
        for c in range(ncols):
            img = Image.open(imgfiles_arr[r,c])
            width, height = img.size
            if(c==0): # y axis
                left=0
                right=width-6.25
                if(r==0): # 500au scale
                    new_x = 435-419
                else: # 4000au scale
                    new_x = 0
            else: # no y axis
                left=7.25
                new_x=c*side+(leftax)
                right=width-6.25

            if(r==nrows-1): # x axis
                bottom=height
            else: 
                bottom=height
            top=7
            new_y = r*side + header

            img = img.crop((left, top, right, bottom))
            img = img.resize((img.width*300/231), (img.height*300/231))
            allimage.paste(img, ((new_x, new_y)))
    display(allimage)
    
    imcbar = Image.open(VICO_loc+'/PaperImages/zoom_images/cbar_vertical.png')
    
    imnew = Image.new('RGBA', (allimage.width+imcbar.width, allimage.height),
                 color='white')
    imnew.paste(allimage, (0,0))
    imnew.paste(imcbar, (allimage.width, int(allimage.height/2 - imcbar.height/2)))

    return imnew


In [ ]:
def display_image(name):
#     temp_img=plt.imread(name)
#     plt.imshow(temp_img)
#     plt.axis('off')
#     plt.margins(x=0, y=0)
    pil_img = DisplayImage(filename=name)
    display(pil_img)

In [ ]:
# use Snap004_n for this
# s=0
f=9
# saveloc = VICO_loc+'/PaperImages/zoom_images'
filename = zoom_intensity_pcolormesh(Snap054_n, '', scale, f=f,
                          vmin=vmin_zoom, vmax=vmax_zoom,
                          show_xlabels=False, show_ylabels=False, 
                          show_cbar=True, vertical_cbar=True,
                                    saveloc=saveloc)
cbar_v_img = Image.open(filename)
print(cbar_v_img.size)
width, height = cbar_v_img.size
left = 300
right = width
top = 0
bottom = height
cbar_v_img = cbar_v_img.crop((left, top, right, bottom))
print(cbar_v_img.size)
cbar_v_filename = saveloc+'/cbar_vertical.png'
cbar_v_img.save(cbar_v_filename)
display_image(cbar_v_filename)
print(cbar_v_filename)

In [ ]:
allimage = make_zoom_allimage(imgfiles_arr=zoomfiles)

In [ ]:
def zoom_pil_image(imgfiles, debug=True, head=0, topax=0,):
    """ 
    Build a PIL image using the imgs in imgfiles
    First row should include y-axes and bottom row should include x-axes.
    """
    nrows = len(imgfiles)
    ncols = len(imgfiles[0])
    if debug: print(f"{nrows=}, {ncols=}")

    topleft = Image.open(imgfiles[0, 0])


    # get side lengths
    corner_img = Image.open(imgfiles[nrows-1, 0])
    middle_img = Image.open(imgfiles[0,1])
    side = middle_img.width
    left = corner_img.width - middle_img.width
    bott = corner_img.height - middle_img.height
    if debug: print(f"{corner_img.size=}, {middle_img.size=}")
    if debug: print(f"{side=}, {left=}, {bott=}, {head=}")

    # make allimage
    allimage=Image.new("RGBA", (ncols*side+left, topleft.height+corner_img.height))

    # paste mini images
    yy = head
    for rr in range(nrows):
        xx = 0 
        for cc in range(ncols):
            if debug: print(f"{rr=}, {cc=}, {xx=}, {yy=}")
            img = Image.open(imgfiles[rr,cc])
            if rr == 0 and cc==0: # shift the top left
                shift = corner_img.width - topleft.width
                allimage.paste(img, ((xx+shift, yy)))
            else:
                allimage.paste(img, ((xx, yy)))
            if cc==0: xx += left+side
            else: xx += side
        yy += topleft.height
    
    details = {
        'nrows':nrows, 'ncols':ncols, 'side':side, 'left':left, 'bott':bott, 'head':head
    }
    return allimage, details

In [ ]:
allimage, details = zoom_pil_image(zoomfiles,)
display(allimage)

In [ ]:
cbimg = cbar_v_img.resize((int(1.5*cbar_v_img.width), int(1.5*cbar_v_img.height)))
newimg = plot.img_concat_right(allimage, cbimg)
display(newimg)
newimg.save(VICO_loc+'/figures/intensity_maps/zoom_12msun.png')